<a href="https://colab.research.google.com/github/Bhushan4-tech/DeepLearning-/blob/main/Freiburg_groceries_Datasets_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/grive")

Mounted at /content/grive


- Freiburg groceries dataset
  - 5000 > 25 classes (256x256x3)

- milk
- snacks
- cold drinks
- cereal
- water
- oil

`folder structure:`
data(main folder)
- train(sub folder level 1)
  - class 1(sub folder level 2)
    - files
  - class 2(sub folder level 2)
    - files
  - class 3(sub folder level 2)
    - files
- test(sub folder level 1)
  - class 1(sub folder level 2)
    - files
  - class 2(sub folder level 2)
    - files
  - class 3(sub folder level 2)
    - files

In [ ]:
import os
import cv2
from glob import glob

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [ ]:
path_ = '/content/grive/MyDrive/DL_CNN/Data'
os.listdir(path_)

['Train_', 'Test_']

In [ ]:
os.listdir(os.path.join(path_,'Train_'))

['SPICES',
 'WATER',
 'TEA',
 'OIL',
 'VINEGAR',
 'SODA',
 'PASTA',
 'SUGAR',
 'RICE',
 'TOMATO_SAUCE',
 'JUICE',
 'CHOCOLATE',
 'FISH',
 'COFFEE',
 'JAM',
 'NUTS',
 'CORN',
 'MILK',
 'HONEY',
 'FLOUR',
 'CANDY',
 'CAKE',
 'CHIPS',
 'BEANS',
 'CEREAL']

In [ ]:
os.listdir(os.path.join(path_,'Test_'))

['PASTA',
 'OIL',
 'SODA',
 'RICE',
 'TEA',
 'TOMATO_SAUCE',
 'SPICES',
 'SUGAR',
 'VINEGAR',
 'WATER',
 'COFFEE',
 'CORN',
 'HONEY',
 'FISH',
 'NUTS',
 'JUICE',
 'MILK',
 'CHOCOLATE',
 'FLOUR',
 'JAM',
 'CHIPS',
 'CEREAL',
 'CAKE',
 'BEANS',
 'CANDY']

# Data Augmentation

In [ ]:
# Seting up the configurations

BATCH_SIZE = 32
TARGET_SIZE = (150,150)

# amount of training data
train_path = '/content/grive/MyDrive/DL_CNN/Data/Train_'
val_path = '/content/grive/MyDrive/DL_CNN/Data/Test_'

training_data_count = len(glob(f'{train_path}/*/*.png'))
validation_data_count = len(glob(f'{val_path}/*/*.png'))

training_steps = training_data_count//BATCH_SIZE
validation_steps = validation_data_count//BATCH_SIZE

training_data_count,validation_data_count,training_steps, validation_steps

(4966, 1681, 155, 52)

In [ ]:
# min value for a pixel in an image > 0
# max value for a pixel in an image > 255

In [ ]:
# Working on training data
train_data_generator = ImageDataGenerator(rescale = 1./255.,
                                          width_shift_range = 0.3,
                                          height_shift_range = 0.3,
                                          shear_range = 0.3,
                                          zoom_range = 0.3,
                                          fill_mode = 'nearest',
                                          horizontal_flip = True,
                                          vertical_flip = True)

train_data = train_data_generator.flow_from_directory(directory = train_path,
                                                      batch_size = BATCH_SIZE,
                                                      class_mode = 'categorical',
                                                      target_size = TARGET_SIZE)

Found 4966 images belonging to 25 classes.


In [ ]:
# Working on training data
val_data_generator = ImageDataGenerator(rescale = 1./255.,
                                          width_shift_range = 0.3,
                                          height_shift_range = 0.3,
                                          shear_range = 0.3,
                                          zoom_range = 0.3,
                                          fill_mode = 'nearest',
                                          horizontal_flip = True,
                                          vertical_flip = True)

val_data = val_data_generator.flow_from_directory(directory = val_path,
                                                      batch_size = BATCH_SIZE,
                                                      class_mode = 'categorical',
                                                      target_size = TARGET_SIZE)

Found 1681 images belonging to 25 classes.


# Building the CNN model

In [ ]:
# CNN model > (Convolutional + pooling layers) + flattened_layer + ANN

- Conv layer 1 : 5 filters
  - 1 image : 256x256
  - post Convolution:
    - 5 images : 256x256
    - 256x256x5

  - post pooling:
    - 5 images : 170x170
    - 170x170x5

- Conv layer 2 : 10 filters
  - 5 image : 170x170
  - post Convolution:
    - 50 images : 170x170
    - 170x170x50

  - post pooling:
    - 50 images : 80x80
    - 80x80x50

- Conv layer 3 : 20 filters
  - 50 image : 80x80
  - post Convolution:
    - 1000 images : 80x80
    - 80x80x1000

  - post pooling:
   - 1000 images : 30x30
   - 30x30x1000

- Conv layer 4 : 20 filters
  - 20000 image : 30x30
  - post Convolution:
    - 20000 images : 30x30
    - 30x30x1000

  - post pooling:
    - 20000 images : 6x6
    - 6x6x20000

In [ ]:
# ------------------------------------ Model building
# Instantiating the Sequential model
cnn = tf.keras.Sequential()

# Adding the input layer
cnn.add(tf.keras.layers.Input(shape = TARGET_SIZE.__add__((3,))))

# ------------------- Conv layers
# Adding the first ConvolutionalxPooling layer
cnn.add(tf.keras.layers.Conv2D(filters = 64, 
                               kernel_size = (5,5), 
                               padding = 'same', 
                               activation = 'relu', 
                               kernel_initializer = 'he_uniform'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = (5,5)))

# Adding the second ConvolutionalxPooling layer
cnn.add(tf.keras.layers.Conv2D(filters = 128, 
                               kernel_size = (5,5), 
                               padding = 'same', 
                               activation = 'relu', 
                               kernel_initializer = 'he_uniform'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = (5,5)))

# Adding the flattening layer
cnn.add(tf.keras.layers.Flatten())

# Adding the hidden layers----------------------

# First hidden layer
cnn.add(tf.keras.layers.Dense(units = 1024,
                              activation = 'elu', 
                              kernel_initializer= 'he_uniform'))

# Second hidden layer
cnn.add(tf.keras.layers.Dense(units = 2048,
                              activation = 'elu', 
                              kernel_initializer= 'he_uniform'))

# Third hidden layer
cnn.add(tf.keras.layers.Dense(units = 512,
                              activation = 'elu', 
                              kernel_initializer= 'he_uniform'))

# Adding the final output layer
cnn.add(tf.keras.layers.Dense(units = 25,
                              activation = 'softmax', 
                              kernel_initializer= 'glorot_uniform'))
# ------------------- Fully connected layer :> ANN
# ------------------------------------ Model building

# ------------------------------------ Model compilation
cnn.compile(optimizer = 'rmsprop',
            loss = 'categorical_crossentropy',
            metrics = ['accuracy','Recall'])
# ------------------------------------ Model compilation

In [ ]:
# Summarizing the model. > 
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 128)       204928    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 128)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 1024)              4

In [ ]:
# Generating the calbacks
early_stopper = EarlyStopping(patience = 15)

In [ ]:
history_object = cnn.fit(train_data,
                         epochs = 100,
                         batch_size = BATCH_SIZE,
                         steps_per_epoch = training_steps,
                         validation_data = val_data,
                         validation_batch_size = BATCH_SIZE,
                         validation_steps = validation_steps,
                         use_multiprocessing = True,
                         workers = 30,
                         callbacks = [early_stopper])

Epoch 1/100
155/155 [==============================] - 396s 2s/step - loss: 3.9175 - accuracy: 0.0742 - recall: 0.0024 - val_loss: 3.1940 - val_accuracy: 0.0811 - val_recall: 0.0000e+00
Epoch 2/100
155/155 [==============================] - 54s 326ms/step - loss: 3.1714 - accuracy: 0.0922 - recall: 0.0036 - val_loss: 3.0889 - val_accuracy: 0.1046 - val_recall: 0.0102
Epoch 3/100
155/155 [==============================] - 55s 323ms/step - loss: 3.1646 - accuracy: 0.0993 - recall: 0.0043 - val_loss: 3.1101 - val_accuracy: 0.1064 - val_recall: 0.0138
Epoch 4/100
155/155 [==============================] - 56s 336ms/step - loss: 3.0834 - accuracy: 0.1007 - recall: 0.0061 - val_loss: 2.9805 - val_accuracy: 0.1238 - val_recall: 0.0102
Epoch 5/100
155/155 [==============================] - 54s 326ms/step - loss: 3.0402 - accuracy: 0.1184 - recall: 0.0061 - val_loss: 2.9816 - val_accuracy: 0.1394 - val_recall: 0.0078
Epoch 6/100
155/155 [==============================] - 56s 337ms/step - loss: 